In [2]:
# --- Setup & Imports ---
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- 1. Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import GlobalConfig, DEFAULT_CORR_SPEC, EWMA_CORR_SPEC, outputs_for_model
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target, load_ifo_features,
    load_full_lagged_features, load_rolling_importance # NEUE IMPORTE
)
# --- MODELL-IMPORT ---
from src.models.ET import ForecastModel # Nutzt ExtraTreesRegressor

print('PROJECT_ROOT =', PROJECT_ROOT)
# --- NEU: Pfad zur Feature-Importance-Output-Ordner ---
FI_PATH = PROJECT_ROOT / "outputs" / "feature_importance" /"outputs_no_missing"

# --- NEUER MASTER-SCHALTER ---
# Setze auf False, um die originale "Gleis 1/2" Pipeline (volles FE-Tuning) zu fahren
# Setze auf True, um die "Gleis 3" Pipeline (dynamische FI-Features) zu fahren
USE_DYNAMIC_FI_PIPELINE = True
# -----------------------------

# --- MODELLNAME ---
if USE_DYNAMIC_FI_PIPELINE:
    MODEL_NAME = "extra_trees_dynamic_fi" # Separater Output-Ordner
else:
    MODEL_NAME = "extra_trees" # Originaler Ordner

outputs_for_model(MODEL_NAME)
print(f'Modell {MODEL_NAME} wird getunt.')

# --- 2. Daten laden ---
y = load_target()  # ΔIP with DatetimeIndex
X_ifo = load_ifo_features() # ifo features (nur für Gleis 1/2)

# Lade die Daten für Gleis 3 (optional)
if USE_DYNAMIC_FI_PIPELINE:
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_PATH)
        rolling_imp = load_rolling_importance(base_dir=FI_PATH)
        # Datenabgleich
        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi, X_full_lagged, rolling_imp = y.loc[idx_fi], X_full_lagged.loc[idx_fi], rolling_imp.loc[idx_fi]
        print('Gleis 3-Daten geladen. Shapes:', X_full_lagged.shape, rolling_imp.shape)
    except FileNotFoundError as e:
        print(f"FEHLER: {e}")
        print("Stelle sicher, dass `feature_importance.ipynb` (Jobs 1 & 3) erfolgreich durchgelaufen ist.")
        # Beende das Skript, wenn die Daten fehlen
        raise
else:
    X_full_lagged, rolling_imp = None, None
    idx = y.index.intersection(X_ifo.index)
    y, X_ifo = y.loc[idx], X_ifo.loc[idx]
    print('Gleis 1/2-Daten geladen. Shapes:', X_ifo.shape, y.shape)


# --- 3. Base config (splits & policy as in thesis) ---
def base_cfg() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis") # Lädt Thesis-Splits (180, 240, etc.)
    # Policy Parameter
    cfg.policy_window   = 24
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 3
    return cfg

cfg0 = base_cfg()

# --- 4. Helper für Korrelations-Spezifikation (bleibt gleich) ---
def make_corr_spec(kind: str) -> dict:
    if kind == 'expanding':
        return dict(DEFAULT_CORR_SPEC)
    elif kind == 'ewm':
        return dict(EWMA_CORR_SPEC)
    else:
        raise ValueError("kind must be 'expanding' or 'ewm'")

# --- 5. TUNING-GRID (Logik-Switch) ---

if USE_DYNAMIC_FI_PIPELINE:
    # --- GRID FÜR GLEIS 3 (Nur Modell-HPs) ---
    print("Erstelle HP-Grid für 'Dynamic FI' (Gleis 3)...")
    N_FEATURES_TO_USE = 20 # Anzahl der Top-Features, die wir auswählen

    n_estimators_list = [100, 300, 500]
    max_features_list = ['sqrt', 0.2, 0.4]
    min_samples_leaf_list = [5, 15, 30]
    weighting_options = [
        {"sample_weight_decay": None},
        {"sample_weight_decay": 0.98}
    ]

    def build_model_grid_dynamic_fi():
        hp_grid = []
        for n_est in n_estimators_list:
            for max_feat in max_features_list:
                for min_leaf in min_samples_leaf_list:
                    for weight_hp in weighting_options:
                        hp = {
                            'n_features_to_use': N_FEATURES_TO_USE,
                            'n_estimators': n_est,
                            'max_features': max_feat,
                            'min_samples_leaf': min_leaf,
                            **weight_hp,
                        }
                        hp_grid.append(hp)
        return hp_grid

    model_grid = build_model_grid_dynamic_fi()

else:
    # --- GRID FÜR GLEIS 1/2 (Original-Grid) ---
    print("Erstelle HP-Grid für 'Full FE' (Gleis 1/2)...")

    # A) Feature Engineering Parameter Listen
    corr_options = [
        ("expanding", make_corr_spec("expanding")),
        ("ewm", make_corr_spec("ewm"))
    ]
    lag_candidates_list = [(1, 2, 3, 6, 12)]
    top_k_lags_list = [1] # Reduziert
    use_rm3_list = [True] # Reduziert
    k1_topk_list = [100, 300]
    redundancy_param_list = [0.90] # Reduziert
    dr_options_list = [
        {'dr_method': 'none'},
        {'dr_method': 'pca', 'pca_var_target': 0.95, 'pca_kmax': 50},
        {'dr_method': 'pls', 'pls_components': 8}
    ]

    # B) Extra Trees Hyperparameter Listen
    n_estimators_list = [300]
    max_features_list = ['sqrt', 0.3] # Reduziert
    min_samples_leaf_list = [5, 15] # Reduziert (von [5] im alten Code, um mehr zu testen)

    # C) Target Blocks & Weighting Listen
    target_block_options = [ None, ["AR1"], ["Chronos"], ["TSFresh"] ]
    weighting_options = [ {"sample_weight_decay": None} ] # Reduziert

    # D) Grid zusammensetzen
    def build_model_grid_optimized():
        hp_grid = []
        for corr_tag, corr_spec in corr_options: # 2
            for lags in lag_candidates_list: # 1
                for k_lags in top_k_lags_list: # 1
                    for rm3 in use_rm3_list: # 1
                        for k1 in k1_topk_list: # 2
                            for red in redundancy_param_list: # 1
                                for dr_opt in dr_options_list: # 3
                                    # Einschränkung
                                    if k1 == 100 and dr_opt['dr_method'] != 'none': continue
                                    # Total FE/DR = (100,none) + (300,none) + (300,pca) + (300,pls) = 4

                                    for n_est in n_estimators_list: # 1
                                        for max_feat in max_features_list: # 2
                                            for min_leaf in min_samples_leaf_list: # 2
                                                for block_set in target_block_options: # 4
                                                    for weight_hp in weighting_options: # 1
                                                        hp = {
                                                            # FE Params
                                                            'lag_candidates': lags,
                                                            'top_k_lags_per_feature': k_lags,
                                                            'use_rm3': rm3,
                                                            'k1_topk': k1,
                                                            'redundancy_param': red,
                                                            **dr_opt,
                                                            # ET Params
                                                            'n_estimators': n_est,
                                                            'max_features': max_feat,
                                                            'min_samples_leaf': min_leaf,
                                                            # Other Params
                                                            **weight_hp,
                                                            'corr_tag': corr_tag,
                                                            'corr_spec': corr_spec,
                                                            'target_block_set': block_set
                                                        }
                                                        hp_grid.append(hp)
        return hp_grid

    model_grid = build_model_grid_optimized()

print("Optimierte HP-Kombinationen:", len(model_grid))
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid ist leer")


# --- 6. Stage A/B Lauf (Logik-Switch) ---
if model_grid:
    if USE_DYNAMIC_FI_PIPELINE:
        # --- LAUF FÜR GLEIS 3 ---
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo, # Platzhalter, wird nicht verwendet
            y=y_fi,
            cfg=cfg0,
            keep_top_k_final=min(5, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//4),
            # NEUE ARGUMENTE
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp
        )

        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo, # Platzhalter, wird nicht verwendet
            y=y_fi,
            cfg=cfg0,
            # NEUE ARGUMENTE
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp
        )
    else:
        # --- LAUF FÜR GLEIS 1/2 (Original) ---
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,
            y=y,
            cfg=cfg0,
            keep_top_k_final=min(5, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//4),
            # (Keine extra Argumente)
        )

        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,
            y=y,
            cfg=cfg0,
            # (Keine extra Argumente)
        )
else:
    print("Keine gültigen HP-Kombinationen gefunden, Stages übersprungen.")

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")

PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Modell extra_trees_dynamic_fi wird getunt.
INFO in load_ifo_features: Renaming columns to ensure validity.
Gleis 3-Daten geladen. Shapes: (407, 2160) (407, 2160)
Erstelle HP-Grid für 'Dynamic FI' (Gleis 3)...
Optimierte HP-Kombinationen: 54
Erstes HP-Set: {'n_features_to_use': 20, 'n_estimators': 100, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'sample_weight_decay': None}
[Stage A] Using DYNAMIC FI pipeline (Gleis 3).
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=54
  - Config 1/54
    · Month 5/20 processed | running...RMSE=1.7381
    · Month 10/20 processed | running...RMSE=1.4005
    · Month 15/20 processed | running...RMSE=1.3165
    · Month 20/20 processed | running...RMSE=1.2315
  - Config 2/54
    · Month 5/20 processed | running...RMSE=1.7815
    · Month 10/20 processed | running...RMSE=1.4469
    · Month 15/20 processed | running...RMSE=1.3498
    · Month 20/20 processed | running...RMSE=1.2461
  - 